# 라이브러리 설치 및 임포트, 시드 고정

In [ ]:
cd ../../..

In [2]:
!pip install -r requirements.txt \
  --extra-index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 206.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 265.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 81.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 159.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 216.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 232.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 217.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 202.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 197.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [3]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, TrainerCallback
from transformers import pipeline
import torch
from peft import LoraConfig, TaskType, get_peft_model
from sklearn.metrics import roc_auc_score
import datetime as dt
import random
import re
import os
from tqdm import tqdm
from torch.utils.data import DataLoader

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED) # Seed 고정

# 데이터 불러오기

In [ ]:
# 전체 fold 파일 경로 리스트 (0~4)
val_fold_idx = 1  # 예: 3을 넣으면 fold3.csv가 validation으로, 나머지(0,1,2,4)가 train으로 사용


fold_paths = [f"./data/kfold_csv/fold{i}.csv" for i in range(4)]

FOLD_VAL   = fold_paths[val_fold_idx]
FOLD_TRAIN = [path for idx, path in enumerate(fold_paths) if idx != val_fold_idx]

print("▶ Train folds:", FOLD_TRAIN)
print("▶ Validation fold:", FOLD_VAL)

TEST_CSV        = "./data/kfold_csv/test_preprocessed.csv"
SUBMISSION_CSV  = "./data/kfold_csv/sample_submission.csv"

▶ Train folds: ['./fold0.csv', './fold2.csv', './fold3.csv']
▶ Validation fold: ./fold1.csv


In [ ]:
# 학습용 데이터프레임
train_df = pd.concat(
    [pd.read_csv(p, encoding="utf-8-sig") for p in FOLD_TRAIN],
    ignore_index=True
)

# 검증용 데이터프레임
val_df   = pd.read_csv(FOLD_VAL, encoding="utf-8-sig")

# ── 필요 없는 열 제거 & 컬럼명 통일 ──
train_df = train_df[['full_text', 'generated']].rename(
    columns={'full_text':'text', 'generated':'label'}
)
val_df   = val_df  [['full_text', 'generated']].rename(
    columns={'full_text':'text', 'generated':'label'}
)

# ── 학습 세트 셔플 ──
train_df = train_df.sample(frac=1, random_state=SEED).reset_index(drop=True)

print("최종 학습 샘플 수:", len(train_df))
print("최종 학습 클래스 분포:", train_df['label'].value_counts().to_dict())
print("검증 샘플 수:", len(val_df))
print("검증 클래스 분포:", val_df['label'].value_counts().to_dict())

최종 학습 샘플 수: 91143
최종 학습 클래스 분포: {1: 45572, 0: 45571}
검증 샘플 수: 30381
검증 클래스 분포: {0: 15191, 1: 15190}


In [7]:
# ==============================================================
# 2) Hugging Face Dataset 변환
# ==============================================================
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)

In [8]:
# ==============================================================
# 3) 토큰화
# ==============================================================

MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-32B-Instruct"  # 사전학습 모델 이름 (Hugging Face 모델 허브)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# 학습/검증 데이터를 토큰화
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# 토크나이저가 반환한 컬럼과 원본 텍스트 컬럼 정리 (모델 입력에 필요 없는 컬럼 제거)
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])

# 라벨 컬럼명 변경
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset   = val_dataset.rename_column("label", "labels")

Map: 100%|██████████| 30381/30381 [00:02<00:00, 13525.59 examples/s]


In [9]:
# ==============================================================
# 4) Data Collator
# ==============================================================
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [10]:
# 장치 설정 (GPU 사용 가능 여부)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # A100에서는 bfloat16 사용 권장
    bnb_4bit_quant_type="nf4",             # NF4 양자화 방식
    bnb_4bit_use_double_quant=True         # 메모리 효율 추가 향상 옵션
)

# 사전훈련 모델 로드 (시퀀스 분류용 헤드 포함) 및 GPU 이동
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, quantization_config=bnb_config, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to(device)

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-32B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-32B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100%|██████████| 27/27 [00:55<00:00,  2.07s/it]
Some weights of ExaoneForSequenceClassification were not initialized from the model checkpoint at LGAI-EXAONE/EXAONE-3.5-32B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ExaoneForSequenceClassification(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 5120, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-63): 64 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSdpaAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=False)
            (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
            (out_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear4bit(in_features=5120, out_features=27392, bias=False)
          (c_fc_1): Linear4bit(in_features=5120, out_features=27392, bias=False)
          (c_proj): Linear4bit(in_features=27392, out_featur

In [11]:
# LoRA 설정 구성
R = 32
LORA_ALPHA = 16
LORA_DROPOUT = 0.1
lora_config = LoraConfig(
    r=R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    task_type=TaskType.SEQ_CLS,
    target_modules= ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

# 원본 모델에 LoRA 어댑터 추가
model = get_peft_model(model, lora_config)


In [12]:
model.print_trainable_parameters()

trainable params: 46,147,584 || all params: 31,525,069,824 || trainable%: 0.1464


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = logits[:, 1]  # 클래스 1의 확률 추정값
    roc_auc = roc_auc_score(labels, probs)
    return {"roc_auc": roc_auc}

In [ ]:
# 훈련 파라미터 설정
training_args = TrainingArguments(
    output_dir="./train&inference/xone/fold1/xone_model1_checkpoint",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    save_strategy="epoch",
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    logging_strategy="steps",
    logging_steps=1000,
    logging_first_step=True,
    save_total_limit=2,
    seed=SEED,
    dataloader_drop_last=False,
    report_to="none",
    label_names=["labels"]
)

In [15]:
# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_102/3656470012.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
# 모델 훈련 시작
trainer.train()

Step,Training Loss
1,1.679700
1000,0.896600
2000,0.570100
3000,0.541500
4000,0.529100
5000,0.527900
6000,0.524300
7000,0.514900
8000,0.515200
9000,0.521000


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# fine-tuned 모델을 로컬에 저장
output_dir = "./train&inference/xone/fold1/xone_model1"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("모델이 저장되었습니다:", output_dir)


모델이 저장되었습니다: ./xone_model1


# TEST 데이터셋 추론

In [19]:
# 테스트 데이터 불러오기
test_df = pd.read_csv(TEST_CSV, encoding='utf-8-sig')
submission_df = pd.read_csv(SUBMISSION_CSV, encoding='utf-8-sig')

print("테스트 샘플 수:", len(test_df))
# 각 테스트 샘플에 대해 추론
pred_probs = []

테스트 샘플 수: 1962


In [20]:
trainer.model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): ExaoneForSequenceClassification(
      (transformer): ExaoneModel(
        (wte): Embedding(102400, 5120, padding_idx=0)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-63): 64 x ExaoneBlock(
            (ln_1): ExaoneRMSNorm()
            (attn): ExaoneAttention(
              (attention): ExaoneSdpaAttention(
                (rotary): ExaoneRotaryEmbedding()
                (k_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=5120, out_features=1024, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=5120, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=32, out_features=1024, bias=

In [ ]:
# 추론 파이프라인 구성 (GPU 사용, 모든 클래스 점수 출력)
clf = pipeline(
    "text-classification",
    model=trainer.model, # 미세조정된 모델
    tokenizer=tokenizer,
    return_all_scores=True,
)

Device set to use cuda:0
/opt/conda/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [22]:
print("샘플 결과 예시:", clf(test_df['paragraph_text'][0]))


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


샘플 결과 예시: [[{'label': 'LABEL_0', 'score': 0.44746026396751404}, {'label': 'LABEL_1', 'score': 0.5525397062301636}]]


In [ ]:
for text in test_df['paragraph_text']:
    scores = clf(text)[0]
    prob_ai = None
    for s in scores:
        if s['label'] in ['LABEL_1', '1', 'generated']:
            prob_ai = s['score']
            break
    if prob_ai is None:
        prob_ai = scores[1]['score']
    pred_probs.append(prob_ai)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [24]:
# 결과를 제출 데이터프레임에 기록
submission_df['generated'] = pred_probs

In [25]:
submission_df

,ID,generated
0,TEST_0000,0.552540
1,TEST_0001,0.993950
2,TEST_0002,0.280616
3,TEST_0003,0.992244
4,TEST_0004,0.941439
...,...,...
1957,TEST_1957,0.996378
1958,TEST_1958,0.998390
1959,TEST_1959,0.435902
1960,TEST_1960,0.281406


In [ ]:
submission_df.to_csv("./ensemble/data/test_ensemble_folding/test_xone_fold1.csv", index=False, encoding="utf-8-sig")

# VAL 데이터셋 배치 추론

In [28]:
def tokenize_test(batch):
    return tokenizer(batch["text"], truncation=True)

val_ds = Dataset.from_pandas(val_df)

val_ds = val_ds.map(tokenize_test, batched=True,
                      remove_columns=["text", "label"])

Map: 100%|██████████| 30381/30381 [00:02<00:00, 13396.13 examples/s]


In [29]:
def collate(features):
    """
    • 동적 padding → tensor 변환
    • tokenizer가 추가한 'length' 류 메타키 제거
    """
    batch = data_collator(features)
    return batch

In [30]:
BATCH_TEST = 8
loader = DataLoader(
    val_ds,
    batch_size=BATCH_TEST,
    shuffle=False,
    collate_fn=collate,
    pin_memory=True,
)

probs_list = []

with torch.no_grad():
    for batch in tqdm(loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = trainer.model(**batch).logits
        probs  = torch.softmax(logits, dim=-1)[:, 1]
        probs_list.append(probs.cpu())

probs = torch.cat(probs_list).to(torch.float32).numpy()
print(f"[✓] Inference done – {len(probs)} samples")

 79%|███████▉  | 2999/3798 [36:28<09:14,  1.44it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
val_df['generated'] = probs
val_df['ID'] = pd.read_csv(FOLD_VAL, encoding="utf-8-sig")['id']
val_df = val_df[['ID', 'generated', 'label']]
val_df.to_csv("./ensemble/data/val_ensemble_folding/val_xone_fold1.csv", index=False, encoding="utf-8-sig")